In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

from cc3_preprocessor import Preprocessor

np.random.seed(42)

In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor(smoothing_factor=5, mode = 'catboost')

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [5]:
y_train.value_counts(normalize=True)

subrogation
0.0    0.77141
1.0    0.22859
Name: proportion, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

subrogation
0.0    0.771296
1.0    0.228704
Name: proportion, dtype: float64

In [7]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Fit complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


## Vanilla CatBoost Model (Default Parameters)

In [8]:
cb_clf = cb.CatBoostClassifier(
    objective='Logloss',
    random_state=42,
    thread_count=-1
)

In [9]:
cat_feature_names = pre.cat_for_encoding_
cb_clf.fit(X_train_proc, y_train, 
           cat_features=cat_feature_names,
           verbose=False)

In [10]:
test_probabilities = cb_clf.predict_proba(X_test_proc)[:, 1]

test_classes = cb_clf.predict(X_test_proc)

print(f"Accuracy: {accuracy_score(y_test, test_classes)}")
print(f"F1 Score: {f1_score(y_test, test_classes)}")
print(f"ROC AUC Score: {roc_auc_score(y_test, test_probabilities)}") # Use probabilities
print(f"PR AUC (Average Precision): {average_precision_score(y_test, test_probabilities)}") # Use probabilities
print(f"Precision: {precision_score(y_test, test_classes)}")
print(f"Recall: {recall_score(y_test, test_classes)}")

Accuracy: 0.8137037037037037
F1 Score: 0.5204957102001907
ROC AUC Score: 0.8388279036310881
PR AUC (Average Precision): 0.6048398830102898
Precision: 0.6326767091541136
Recall: 0.4421052631578947


## CatBoost with Optuna Tuning

In [11]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
CAT_FEATURES = pre.cat_for_encoding_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season']


In [13]:
def objective(trial: optuna.trial.Trial) -> float:

    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True), 
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'early_stopping_rounds': 100
    }

    params['eval_metric'] = 'Logloss'
    
    model = CatBoostClassifier(**params)
    
    model.fit(
        X_train_proc, y_train,
        eval_set=(X_test_proc, y_test),
        cat_features=CAT_FEATURES,
        verbose=False
    )

    y_preds = model.predict(X_test_proc)
    
    manual_f1_score = f1_score(y_test, y_preds, pos_label=1)
    
    return manual_f1_score

In [14]:
print("\n2. Starting Optuna study...")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

study.optimize(
    objective, 
    n_trials=50, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-06 16:51:04,364] A new study created in memory with name: no-name-3b3c473c-dec9-4d7d-acdf-aa8130888e31



2. Starting Optuna study...


Best trial: 0. Best value: 0.542583:   2%|▏         | 1/50 [00:03<02:35,  3.17s/it]

[I 2025-11-06 16:51:07,541] Trial 0 finished with value: 0.5425834843385969 and parameters: {'learning_rate': 0.1074605335296081, 'depth': 9, 'l2_leaf_reg': 0.0842528281646853, 'subsample': 0.9, 'random_strength': 1.29522223590442e-07, 'bagging_temperature': 0.01173931162448183, 'border_count': 67, 'scale_pos_weight': 6.1509213046462206}. Best is trial 0 with value: 0.5425834843385969.


Best trial: 0. Best value: 0.542583:   4%|▍         | 2/50 [00:05<01:56,  2.42s/it]

...

[I 2025-11-06 16:53:14,864] Trial 47 finished with value: 0.5914519170333123 and parameters: {'learning_rate': 0.014633875705546834, 'depth': 4, 'l2_leaf_reg': 0.10813022243510685, 'subsample': 1.0, 'random_strength': 0.012020955572240392, 'bagging_temperature': 0.2318671644605746, 'border_count': 79, 'scale_pos_weight': 3.0006544383523335}. Best is trial 42 with value: 0.5998608211551845.


Best trial: 42. Best value: 0.599861:  98%|█████████▊| 49/50 [02:15<00:04,  4.40s/it]

[I 2025-11-06 16:53:19,495] Trial 48 finished with value: 0.5991792065663475 and parameters: {'learning_rate': 0.01210834418906949, 'depth': 3, 'l2_leaf_reg': 0.6681242579579779, 'subsample': 0.9, 'random_strength': 0.004586516189444487, 'bagging_temperature': 0.359271273528436, 'border_count': 234, 'scale_pos_weight': 2.3158167619906744}. Best is trial 42 with value: 0.5998608211551845.


Best trial: 42. Best value: 0.599861: 100%|██████████| 50/50 [02:23<00:00,  2.87s/it]

[I 2025-11-06 16:53:27,879] Trial 49 finished with value: 0.5761865112406328 and parameters: {'learning_rate': 0.01231754707003587, 'depth': 7, 'l2_leaf_reg': 2.495804348361054, 'subsample': 0.9, 'random_strength': 0.003370790234784927, 'bagging_temperature': 0.4528347561753193, 'border_count': 228, 'scale_pos_weight': 1.469804042684729}. Best is trial 42 with value: 0.5998608211551845.

Optuna study finished.
Number of finished trials: 50

Best trial:
  Value (Max F1 Score): 0.5999
  Best Hyperparameters:
    learning_rate: 0.019223480258071995
    depth: 3
    l2_leaf_reg: 0.13165643382999032
    subsample: 0.5
    random_strength: 0.037315270637859124
    bagging_temperature: 0.2311472336929682
    border_count: 68
    scale_pos_weight: 2.17476792382015


In [15]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 2000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 50 # Keep early stopping
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.019223480258071995, 'depth': 3, 'l2_leaf_reg': 0.13165643382999032, 'subsample': 0.5, 'random_strength': 0.037315270637859124, 'bagging_temperature': 0.2311472336929682, 'border_count': 68, 'scale_pos_weight': 2.17476792382015}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.5999

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.90      0.81      0.85      4165
   Class 1.0       0.53      0.70      0.60      1235

    accuracy                           0.79      5400
   macro avg       0.71      0.76      0.73      5400
weighted avg       0.82      0.79      0.80      5400



In [16]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            1
2       4655051            0
3       6728725            1
4       9848460            1


In [17]:
prediction.to_csv("results/catboost_prediction.csv", index=False)

In [18]:
importances = best_model.get_feature_importance()
feature_names = best_model.feature_names_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("--- Feature Importance Analysis ---")
print(f"Total features: {len(feature_importance_df)}")

print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

print("\nBottom 10 Least Important Features:")
print(feature_importance_df.tail(10))

n_features_to_remove = 100
n_features_to_keep = len(feature_importance_df) - n_features_to_remove
top_features = feature_importance_df.head(n_features_to_keep)['feature'].tolist()

print(f"\nKeeping top {len(top_features)} features and removing bottom {n_features_to_remove}.")

X_train_top_features = X_train_proc[top_features]
X_test_top_features = X_test_proc.reindex(columns=top_features, fill_value=0) 

original_cat_features = set(CAT_FEATURES)
top_features_set = set(top_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

print(f"Original categorical features: {len(CAT_FEATURES)}")
print(f"Categorical features kept: {len(new_cat_features)}")

best_params_from_optuna = study.best_trial.params

final_params_new = best_params_from_optuna.copy()
final_params_new.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 50
})

print("\nRetraining model with top features...")
new_model = CatBoostClassifier(**final_params_new)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

print("\n--- Model Performance Comparison ---")
# 'final_f1' comes from cell 30 in your notebook
print(f"Original F1 score (all features): {final_f1:.4f}")
print(f"New F1 score (top {len(top_features)} features): {new_f1:.4f}")

print("\nNew Model Classification Report (Top Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))

--- Feature Importance Analysis ---
Total features: 136

Top 10 Most Important Features:
                        feature  importance
44              liab_x_multicar   10.137100
135  recovery_feasibility_score    8.811156
55                 liab_inverse    7.591556
54                liab_prct_log    6.934579
13                    liab_prct    6.795418
50          multicar_x_highrisk    5.022059
51            liab_prct_squared    4.322328
52              liab_prct_cubed    4.291416
58     is_multi_vehicle_unclear    4.088062
56         liab_inverse_squared    4.059979

Bottom 10 Least Important Features:
                feature  importance
101        large_payout         0.0
102       medium_payout         0.0
103        small_payout         0.0
104   very_large_payout         0.0
70     parking_accident         0.0
107     moderate_damage         0.0
108        minor_damage         0.0
110  can_afford_vehicle         0.0
66      evidence_strong         0.0
64        evidence_none       